In [37]:
function rosenbrock(x, y)
    return (1.0 - x)^2 + 100.0 * (y - x^2)^2
end

function rastrigin(x, y)
    return 20 + x^2 + y^2 - 10 * (cos(2π * x) + cos(2π * y))
end

function shvefel(x, y)
    return 418.9829*2 - (x*sin(sqrt(abs(x))) + y*sin(sqrt(abs(y))))
end

using Plots
using Random

# Функция двух переменных
f(x, y) = rosenbrock(x, y)

# Параметры алгоритма
initial_pop_size = 50
n_generations = 100

x_range = (0.0, 3.0)
y_range = (0.0, 3.0)

tournament_size = 3
max_age = 10

# Описание особи
mutable struct Indiv
    x::Float64
    y::Float64
    age::Int
    new_this_generation::Bool
end

# Турнирная селекция
function tournament_selection(population, fitness_values, tournament_size)
    indices = rand(1:length(population), tournament_size)
    winner_idx = indices[argmin([fitness_values[i] for i in indices])]
    return population[winner_idx]
end

# Линейное скрещивание
function linear_crossover(parent1, parent2, x_range, y_range)
    alpha = rand()
    beta = 1 - alpha

    child_x = alpha * parent1.x + beta * parent2.x
    child_y = alpha * parent1.y + beta * parent2.y

    child_x = clamp(child_x, x_range[1], x_range[2])
    child_y = clamp(child_y, y_range[1], y_range[2])

    return (child_x, child_y)
end

population = [Indiv(
    rand() * (x_range[2] - x_range[1]) + x_range[1],
    rand() * (y_range[2] - y_range[1]) + y_range[1],
    0,
    false
) for _ in 1:initial_pop_size]

best_xy_per_gen = []
best_f_per_gen = []
pop_size_per_gen = Int[]
all_populations = []

push!(all_populations, deepcopy(population))

# Основной цикл
for generation in 1:n_generations
    for ind in population
        ind.age += 1
        ind.new_this_generation = false
    end

    fitness = [f(ind.x, ind.y) for ind in population]
    best_idx = argmin(fitness)
    best_ind = population[best_idx]

    push!(best_xy_per_gen, (best_ind.x, best_ind.y))
    push!(best_f_per_gen, fitness[best_idx])
    push!(pop_size_per_gen, length(population))

    # Удаление старых особей
    filter!(ind -> ind.age < max_age, population)

    new_Indivs = []
    n_offspring = 5

    for _ in 1:n_offspring
        fitness = [f(ind.x, ind.y) for ind in population]
        parent1 = tournament_selection(population, fitness, tournament_size)
        parent2 = tournament_selection(population, fitness, tournament_size)

        if rand() < 0.7
            child_x, child_y = linear_crossover(parent1, parent2, x_range, y_range)
        else
            if rand(Bool)
                child_x, child_y = parent1.x, parent1.y
            else
                child_x, child_y = parent2.x, parent2.y
            end
        end

        # Мутация
        if rand() < 0.1
            child_x += randn() * 0.5
            child_y += randn() * 0.5
            child_x = clamp(child_x, x_range[1], x_range[2])
            child_y = clamp(child_y, y_range[1], y_range[2])
        end

        push!(new_Indivs, Indiv(child_x, child_y, 0, true))
    end

    append!(population, new_Indivs)
    push!(all_populations, deepcopy(population))
end

# Подготовка для визуализации
xs = range(x_range[1], x_range[2], length=100)
ys = range(y_range[1], y_range[2], length=100)
zs = [f(x, y) for x in xs, y in ys]

anim = @animate for i in 1:n_generations
    current_pop = all_populations[i]

    # Контурный график
    p1 = contour(xs, ys, zs',
        title = "Поколение $(i) - контурный график",
        xlabel = "x",
        ylabel = "y",
        fill = true,
        levels = 30,
        color = :viridis,
        legend = :bottomright)

    # 3D график
    p2 = surface(xs, ys, zs',
        title = "Поколение $(i) - 3D вид",
        xlabel = "x",
        ylabel = "y",
        zlabel = "f(x,y)",
        color = :viridis,
        alpha = 0.7,
        camera = (30, 30))

    # Визуализация особей на контурном графике
    existing = filter(ind -> !ind.new_this_generation, current_pop)
    new_inds = filter(ind -> ind.new_this_generation, current_pop)

    scatter!(p1, [ind.x for ind in existing], [ind.y for ind in existing],
             markersize = 4, alpha = 0.4, color = :grey, label = "Старые")

    scatter!(p1, [ind.x for ind in new_inds], [ind.y for ind in new_inds],
             markersize = 5, color = :red, label = "Новые")

    best_xy = best_xy_per_gen[i]
    scatter!(p1, [best_xy[1]], [best_xy[2]],
             color = :green, markershape = :star, markersize = 7, label = "Лучший")
    
    # Визуализация особей на 3D графике
    for ind in existing
        z_val = f(ind.x, ind.y)
        scatter!(p2, [ind.x], [ind.y], [z_val],
                markersize = 3, alpha = 0.4, color = :grey, label = "")
    end
    
    for ind in new_inds
        z_val = f(ind.x, ind.y)
        scatter!(p2, [ind.x], [ind.y], [z_val],
                markersize = 4, color = :red, label = "")
    end
    
    best_z = f(best_xy[1], best_xy[2])
    scatter!(p2, [best_xy[1]], [best_xy[2]], [best_z],
             color = :green, markershape = :star, markersize = 5, label = "")
    
    # Компоновка двух графиков рядом
    plot(p1, p2, layout = (1, 2), size = (1200, 600))
end

gif(anim, "ga_2d_rosen.gif", fps=10)

println("Итоговый размер популяции: $(length(population))")

Итоговый размер популяции: 50


┌ Info: Saved animation to /Users/d.okutin/ga_2d_rosen.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


In [38]:
function rosenbrock(x, y)
    return (1.0 - x)^2 + 100.0 * (y - x^2)^2
end

function rastrigin(x, y)
    return 20 + x^2 + y^2 - 10 * (cos(2π * x) + cos(2π * y))
end

function shvefel(x, y)
    return 418.9829*2 - (x*sin(sqrt(abs(x))) + y*sin(sqrt(abs(y))))
end

using Plots
using Random

# Функция двух переменных
f(x, y) = shvefel(x, y)

# Параметры алгоритма
initial_pop_size = 50
n_generations = 100

x_range = (0.0, 3.0)
y_range = (0.0, 3.0)

tournament_size = 3
max_age = 10

population = [Indiv(
    rand() * (x_range[2] - x_range[1]) + x_range[1],
    rand() * (y_range[2] - y_range[1]) + y_range[1],
    0,
    false
) for _ in 1:initial_pop_size]

best_xy_per_gen = []
best_f_per_gen = []
pop_size_per_gen = Int[]
all_populations = []

push!(all_populations, deepcopy(population))

# Основной цикл
for generation in 1:n_generations
    for ind in population
        ind.age += 1
        ind.new_this_generation = false
    end

    fitness = [f(ind.x, ind.y) for ind in population]
    best_idx = argmin(fitness)
    best_ind = population[best_idx]

    push!(best_xy_per_gen, (best_ind.x, best_ind.y))
    push!(best_f_per_gen, fitness[best_idx])
    push!(pop_size_per_gen, length(population))

    # Удаление старых особей
    filter!(ind -> ind.age < max_age, population)

    new_Indivs = []
    n_offspring = 5

    for _ in 1:n_offspring
        fitness = [f(ind.x, ind.y) for ind in population]
        parent1 = tournament_selection(population, fitness, tournament_size)
        parent2 = tournament_selection(population, fitness, tournament_size)

        if rand() < 0.7
            child_x, child_y = linear_crossover(parent1, parent2, x_range, y_range)
        else
            if rand(Bool)
                child_x, child_y = parent1.x, parent1.y
            else
                child_x, child_y = parent2.x, parent2.y
            end
        end

        # Мутация
        if rand() < 0.1
            child_x += randn() * 0.5
            child_y += randn() * 0.5
            child_x = clamp(child_x, x_range[1], x_range[2])
            child_y = clamp(child_y, y_range[1], y_range[2])
        end

        push!(new_Indivs, Indiv(child_x, child_y, 0, true))
    end

    append!(population, new_Indivs)
    push!(all_populations, deepcopy(population))
end

# Подготовка для визуализации
xs = range(x_range[1], x_range[2], length=100)
ys = range(y_range[1], y_range[2], length=100)
zs = [f(x, y) for x in xs, y in ys]

anim = @animate for i in 1:n_generations
    current_pop = all_populations[i]

    # Контурный график
    p1 = contour(xs, ys, zs',
        title = "Поколение $(i) - контурный график",
        xlabel = "x",
        ylabel = "y",
        fill = true,
        levels = 30,
        color = :viridis,
        legend = :bottomright)

    # 3D график
    p2 = surface(xs, ys, zs',
        title = "Поколение $(i) - 3D вид",
        xlabel = "x",
        ylabel = "y",
        zlabel = "f(x,y)",
        color = :viridis,
        alpha = 0.7,
        camera = (30, 30))

    # Визуализация особей на контурном графике
    existing = filter(ind -> !ind.new_this_generation, current_pop)
    new_inds = filter(ind -> ind.new_this_generation, current_pop)

    scatter!(p1, [ind.x for ind in existing], [ind.y for ind in existing],
             markersize = 4, alpha = 0.4, color = :grey, label = "Старые")

    scatter!(p1, [ind.x for ind in new_inds], [ind.y for ind in new_inds],
             markersize = 5, color = :red, label = "Новые")

    best_xy = best_xy_per_gen[i]
    scatter!(p1, [best_xy[1]], [best_xy[2]],
             color = :green, markershape = :star, markersize = 7, label = "Лучший")
    
    # Визуализация особей на 3D графике
    for ind in existing
        z_val = f(ind.x, ind.y)
        scatter!(p2, [ind.x], [ind.y], [z_val],
                markersize = 3, alpha = 0.4, color = :grey, label = "")
    end
    
    for ind in new_inds
        z_val = f(ind.x, ind.y)
        scatter!(p2, [ind.x], [ind.y], [z_val],
                markersize = 4, color = :red, label = "")
    end
    
    best_z = f(best_xy[1], best_xy[2])
    scatter!(p2, [best_xy[1]], [best_xy[2]], [best_z],
             color = :green, markershape = :star, markersize = 5, label = "")
    
    # Компоновка двух графиков рядом
    plot(p1, p2, layout = (1, 2), size = (1200, 600))
end

gif(anim, "ga_2d_swefel.gif", fps=10)

println("Итоговый размер популяции: $(length(population))")

Итоговый размер популяции: 50


┌ Info: Saved animation to /Users/d.okutin/ga_2d_swefel.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


In [39]:
function rosenbrock(x, y)
    return (1.0 - x)^2 + 100.0 * (y - x^2)^2
end

function rastrigin(x, y)
    return 20 + x^2 + y^2 - 10 * (cos(2π * x) + cos(2π * y))
end

function shvefel(x, y)
    return 418.9829*2 - (x*sin(sqrt(abs(x))) + y*sin(sqrt(abs(y))))
end

using Plots
using Random

# Функция двух переменных
f(x, y) = rastrigin(x, y)

# Параметры алгоритма
initial_pop_size = 50
n_generations = 100

x_range = (0.0, 3.0)
y_range = (0.0, 3.0)

tournament_size = 3
max_age = 10

population = [Indiv(
    rand() * (x_range[2] - x_range[1]) + x_range[1],
    rand() * (y_range[2] - y_range[1]) + y_range[1],
    0,
    false
) for _ in 1:initial_pop_size]

best_xy_per_gen = []
best_f_per_gen = []
pop_size_per_gen = Int[]
all_populations = []

push!(all_populations, deepcopy(population))

# Основной цикл
for generation in 1:n_generations
    for ind in population
        ind.age += 1
        ind.new_this_generation = false
    end

    fitness = [f(ind.x, ind.y) for ind in population]
    best_idx = argmin(fitness)
    best_ind = population[best_idx]

    push!(best_xy_per_gen, (best_ind.x, best_ind.y))
    push!(best_f_per_gen, fitness[best_idx])
    push!(pop_size_per_gen, length(population))

    # Удаление старых особей
    filter!(ind -> ind.age < max_age, population)

    new_Indivs = []
    n_offspring = 5

    for _ in 1:n_offspring
        fitness = [f(ind.x, ind.y) for ind in population]
        parent1 = tournament_selection(population, fitness, tournament_size)
        parent2 = tournament_selection(population, fitness, tournament_size)

        if rand() < 0.7
            child_x, child_y = linear_crossover(parent1, parent2, x_range, y_range)
        else
            if rand(Bool)
                child_x, child_y = parent1.x, parent1.y
            else
                child_x, child_y = parent2.x, parent2.y
            end
        end

        # Мутация
        if rand() < 0.1
            child_x += randn() * 0.5
            child_y += randn() * 0.5
            child_x = clamp(child_x, x_range[1], x_range[2])
            child_y = clamp(child_y, y_range[1], y_range[2])
        end

        push!(new_Indivs, Indiv(child_x, child_y, 0, true))
    end

    append!(population, new_Indivs)
    push!(all_populations, deepcopy(population))
end

# Подготовка для визуализации
xs = range(x_range[1], x_range[2], length=100)
ys = range(y_range[1], y_range[2], length=100)
zs = [f(x, y) for x in xs, y in ys]

anim = @animate for i in 1:n_generations
    current_pop = all_populations[i]

    # Контурный график
    p1 = contour(xs, ys, zs',
        title = "Поколение $(i) - контурный график",
        xlabel = "x",
        ylabel = "y",
        fill = true,
        levels = 30,
        color = :viridis,
        legend = :bottomright)

    # 3D график
    p2 = surface(xs, ys, zs',
        title = "Поколение $(i) - 3D вид",
        xlabel = "x",
        ylabel = "y",
        zlabel = "f(x,y)",
        color = :viridis,
        alpha = 0.7,
        camera = (30, 30))

    # Визуализация особей на контурном графике
    existing = filter(ind -> !ind.new_this_generation, current_pop)
    new_inds = filter(ind -> ind.new_this_generation, current_pop)

    scatter!(p1, [ind.x for ind in existing], [ind.y for ind in existing],
             markersize = 4, alpha = 0.4, color = :grey, label = "Старые")

    scatter!(p1, [ind.x for ind in new_inds], [ind.y for ind in new_inds],
             markersize = 5, color = :red, label = "Новые")

    best_xy = best_xy_per_gen[i]
    scatter!(p1, [best_xy[1]], [best_xy[2]],
             color = :green, markershape = :star, markersize = 7, label = "Лучший")
    
    # Визуализация особей на 3D графике
    for ind in existing
        z_val = f(ind.x, ind.y)
        scatter!(p2, [ind.x], [ind.y], [z_val],
                markersize = 3, alpha = 0.4, color = :grey, label = "")
    end
    
    for ind in new_inds
        z_val = f(ind.x, ind.y)
        scatter!(p2, [ind.x], [ind.y], [z_val],
                markersize = 4, color = :red, label = "")
    end
    
    best_z = f(best_xy[1], best_xy[2])
    scatter!(p2, [best_xy[1]], [best_xy[2]], [best_z],
             color = :green, markershape = :star, markersize = 5, label = "")
    
    # Компоновка двух графиков рядом
    plot(p1, p2, layout = (1, 2), size = (1200, 600))
end

gif(anim, "ga_2d_rast.gif", fps=10)

println("Итоговый размер популяции: $(length(population))")

Итоговый размер популяции: 50


┌ Info: Saved animation to /Users/d.okutin/ga_2d_rast.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156
